# **Notebook de test de Téléchargement de donnée de basket depuis le site nba.com**

Sommaire

## Imports

In [21]:
%load_ext autoreload
%autoreload 2
from TelechargementDonnees import JourneeSiteNba, PasDeMatchError, JoueursSiteNba, DriverFirefox,gererCookie
import pandas as pd
import numpy as np
import time

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException,TimeoutException, ElementClickInterceptedException
from selenium.webdriver.support.ui import Select

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Telecharger les données liees aux matchs depuis le site internet pour une ou plusieurs dates  
1. on telecharge une journee
1. on sauvegarder dans un json

In [ ]:
#telecharger une journee
try : 
    journee=JourneeSiteNba('2020-12-28')
except PasDeMatchError as e : 
    print(e)
#exporter
journee.saveCsv()

In [11]:
# pour un ensemeble de journee : 
for j in ['2021-01-%02d' %n  for n in range(4,11)] : 
    print(j)
    try :
        journee=JourneeSiteNba(j)
        journee.saveCsv()
    except PasDeMatchError as e: 
        print(e)    
        continue

2021-01-04
0 https://fr.global.nba.com/boxscore/#!/0022000093
1 https://fr.global.nba.com/boxscore/#!/0022000094
2 https://fr.global.nba.com/boxscore/#!/0022000095
3 https://fr.global.nba.com/boxscore/#!/0022000096
4 https://fr.global.nba.com/boxscore/#!/0022000097
5 https://fr.global.nba.com/boxscore/#!/0022000098
6 https://fr.global.nba.com/boxscore/#!/0022000099
7 https://fr.global.nba.com/boxscore/#!/0022000100
8 https://fr.global.nba.com/boxscore/#!/0022000101
2021-01-05
0 https://fr.global.nba.com/boxscore/#!/0022000102
1 https://fr.global.nba.com/boxscore/#!/0022000103
2 https://fr.global.nba.com/boxscore/#!/0022000104
3 https://fr.global.nba.com/boxscore/#!/0022000105
4 https://fr.global.nba.com/boxscore/#!/0022000106
2021-01-06
0 https://fr.global.nba.com/boxscore/#!/0022000107
1 https://fr.global.nba.com/boxscore/#!/0022000108
2 https://fr.global.nba.com/boxscore/#!/0022000109
3 https://fr.global.nba.com/boxscore/#!/0022000110
4 https://fr.global.nba.com/boxscore/#!/002200011

## 2. Telecharger les données liees aux joueurs depuis le site internet 
1. on se connecte à la page principale des joueurs
1. on passe de lettre en lettre pour sauvegarder les donnees

In [62]:
#creation de la classe
pagejoueuer=JoueursSiteNba()
#acces aux donnees
pagejoueuer.dfJoueurs
#export des donnees
pagejoueuer.dfJoueurs.reset_index(drop=True).to_json(r'C:\Users\martin.schoreisz\Documents\AffairesEnCours\temp\basket\Joueurs_saison_2020-2021\Joueurs_saison_2020-2021.json')

## 3. Test donnees match sur site americain
1. on se connecte à la page des matchs en fonction de la date
1. on regarde pour telecharger les données
1. on cherche les joueurs
1. on recupere les joueurs inconnus lors de chaque match

In [25]:
driver=DriverFirefox().driver

In [26]:
driver.get('https://www.nba.com/games')
time.sleep(10)

In [27]:
gererCookie(driver)

In [40]:
#par date 
date='2021-01-02'
adresseJournee=f'https://www.nba.com/games?date={date}'
driver.get(adresseJournee)
time.sleep(5)

In [41]:
#recuperer les elements a cliquer
try :
    #containerScore=WebDriverWait(self.driver, 20).until(EC.presence_of_element_located((By.XPATH,f"//div[@class='snapshot-footer']"))) #si besoin que la ligne dessous ne fonctionne pas
    elementsScore=WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.LINK_TEXT, 'BOX SCORE')))  
except TimeoutException :
    raise PasDeMatchError(dateJournee)

In [51]:
#parcourir les elements
for e,p in enumerate([p.get_attribute("href") for p in elementsScore]) : 
    driver.get(e)
    time.sleep(5)
    driver.implicitly_wait(20)
    #sur l'onglet box-score on recupere les stats des equipes
    dfsEquipes=pd.read_html(driver.page_source)
    for i in (0,1) : 
        break

NoSuchElementException: Message: Web element reference not seen before: {"element-6066-11e4-a52e-4f735466cecf":"27ee7a41-542b-4408-accb-c4bebc3a04fe"}


In [10]:
#une fois sur l'onglet Box Score
dfsEquipes=pd.read_html(driver.page_source)
#ensuite avec un for on peut iterer sur les deux df, les mettres en forme (il faut supprimer la derniere lettre du starting 5), trouver les DNP et blesses, etc...

In [12]:
#passer sur l'onglet summary : 
elementSummary=WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.LINK_TEXT, 'Summary')))
elementSummary.click()

In [13]:
#la df du match
dfMatch=pd.read_html(driver.page_source)[0]
#ensuite on peut la mettre en forme comme la partie du site france
#et ensuite on exporte les dfs commme avant

In [15]:
dfMatch

,Unnamed: 0,Q1,Q2,Q3,Q4,FINAL
0,SAC,36,28,17,13,94
1,HOU,36,28,19,19,102


In [55]:
dfsEquipes[0].columns
nomsColonnesStat=['nom', 'minute','tir_reussi','tir_tentes', 'pct_tir', 'trois_pt_r', 'trois_pt_t', 'pct_3_pt','lanc_frc_r', 'lanc_frc_t', 'pct_lfrc',
 'rebonds_o', 'rebonds_d', 'rebonds', 'passes_dec', 'steal','contres','ball_perdu', 'faute_p','points', 'plus_moins']

In [69]:
#mise en forme
df=dfsEquipes[0].copy()
df.drop(df.tail(1).index,inplace=True)
df.columns=nomsColonnesStat
df.loc[df.head(5).index,'nom']=df.head(5).nom.str[:-1]

In [71]:
df.dtypes

nom           object
minute        object
tir_reussi    object
tir_tentes    object
pct_tir       object
trois_pt_r    object
trois_pt_t    object
pct_3_pt      object
lanc_frc_r    object
lanc_frc_t    object
pct_lfrc      object
rebonds_o     object
rebonds_d     object
rebonds       object
passes_dec    object
steal         object
contres       object
ball_perdu    object
faute_p       object
points        object
plus_moins    object
dtype: object